In [1]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import talib
import torch.utils.data as Data

我想要用lstm的网络来预测收盘价。  
x是多日的价格涨跌幅度，都是在-0.1到0.1之间。以及成交量的倍率  
y是收盘价  

# 获取数据

In [2]:
sys.path.append('../../')
import  DataSource
zz500 = DataSource.get_zz500_codes() # 我看中证500的数据
# 我这里查看第一个股票的吧
dt = DataSource.get_data(zz500[0])

In [3]:
dt.head()

,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST
date,,,,,,,,,,,,,
2010-01-04,sh.600004,5.046509,5.111144,5.041537,5.046509,5.046509,15478854,157747809.0,2,1.525010,1,0.0000,0
2010-01-05,sh.600004,5.046509,5.091256,5.041537,5.066397,5.046509,11070922,112683563.0,2,1.090731,1,0.3941,0
2010-01-06,sh.600004,5.056453,5.066397,4.981874,4.991818,5.066397,9410665,95205863.0,2,0.927159,1,-1.4720,0
2010-01-07,sh.600004,4.971930,4.996790,4.827744,4.862548,4.991818,8961813,88611912.0,2,0.882937,1,-2.5896,0
2010-01-08,sh.600004,4.837688,5.096228,4.837688,5.061425,4.862548,14923812,149969456.0,2,1.470326,1,4.0900,0


In [4]:
N = 60 # 多少日的数据
# 我这里仅仅是记录如下的列的涨跌幅度
rate_columns = ['open', 'high', 'low', 'close', 'volume']
input_size = len(rate_columns)
seq_len = N    # 多少个时间序列的股票
output_size = 1 # 我只是输出一个今天的收盘价相比较昨日的涨跌比率
num_layers = 10 # 多少个gru合并
hidden_size = 20 # 隐藏层的宽度
batch_size = 16 # 一个批次有多少个数据
lr = 0.01
# 这里先看看怎样快速的截取数据,然后还要归一化，

In [5]:

for i in rate_columns:
    dt[f'{i}_shift'] = dt[f'{i}'].shift()
    dt[f'{i}_rate'] =  (dt[f'{i}'] - dt[f'{i}_shift'])/dt[f'{i}_shift']

dt2 = dt.loc[:, [f'{i}_rate' for i in rate_columns]]
dt2.head()

,open_rate,high_rate,low_rate,close_rate,volume_rate
date,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.000000,-0.003891,0.000000,0.003941,-0.284771
2010-01-06,0.001970,-0.004883,-0.011834,-0.014720,-0.149966
2010-01-07,-0.016716,-0.013739,-0.030938,-0.025896,-0.047696
2010-01-08,-0.027000,0.019900,0.002060,0.040900,0.665267


In [6]:
# 价格的列名
dt_shift_days = [] # 多少日的数据放在这里边,这里将shift(0)作为第一个了
for i in range(N):
    _dt_shift_day = dt2.shift(i)
    dt_shift_days.append(_dt_shift_day)

In [7]:
dt_shift_days[0].head()

,open_rate,high_rate,low_rate,close_rate,volume_rate
date,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.000000,-0.003891,0.000000,0.003941,-0.284771
2010-01-06,0.001970,-0.004883,-0.011834,-0.014720,-0.149966
2010-01-07,-0.016716,-0.013739,-0.030938,-0.025896,-0.047696
2010-01-08,-0.027000,0.019900,0.002060,0.040900,0.665267


In [8]:
# 然后这个是反序的，所以需要颠倒
dt_shift_days.reverse()

In [9]:
dt_shift_days[-1].head()

,open_rate,high_rate,low_rate,close_rate,volume_rate
date,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.000000,-0.003891,0.000000,0.003941,-0.284771
2010-01-06,0.001970,-0.004883,-0.011834,-0.014720,-0.149966
2010-01-07,-0.016716,-0.013739,-0.030938,-0.025896,-0.047696
2010-01-08,-0.027000,0.019900,0.002060,0.040900,0.665267


In [10]:
# 这里转成numpy
np_shift_days = [i.to_numpy() for i in dt_shift_days]

In [11]:
np_shift_days[-1]

array([[        nan,         nan,         nan,         nan,         nan],
       [ 0.        , -0.00389105,  0.        ,  0.00394089, -0.28477121],
       [ 0.00197044, -0.00488281, -0.01183432, -0.01472031, -0.14996556],
       ...,
       [-0.00319149,  0.00636943,  0.00644468,  0.00106724, -0.24561394],
       [ 0.00747065, -0.00421941, -0.01707577, -0.01385928,  0.08558971],
       [-0.02224576, -0.01483051, -0.00217155,  0.00540541, -0.23457175]])

In [12]:
# 然后进行水平拼接

In [13]:
x_price = np.hstack(np_shift_days)
# 这里需要舍弃一下一些数据。前面的舍弃N个最后舍弃一个
x_price = x_price[N:-1]
x_price

array([[ 0.        , -0.00389105,  0.        , ...,  0.0095986 ,
         0.00599315, -0.07705891],
       [ 0.00197044, -0.00488281, -0.01183432, ..., -0.00259291,
        -0.01021277, -0.28696785],
       [-0.01671583, -0.01373896, -0.03093812, ..., -0.01039861,
        -0.01633706,  0.05892651],
       ...,
       [-0.00720165, -0.00717213, -0.01139896, ..., -0.00746269,
        -0.00319149,  0.22996791],
       [ 0.        ,  0.00515996,  0.00943396, ...,  0.00644468,
         0.00106724, -0.24561394],
       [ 0.00621762,  0.01334702,  0.00726895, ..., -0.01707577,
        -0.01385928,  0.08558971]])

In [14]:
x_price.shape

(3602, 300)

In [15]:
# 这里需要更改维度
x_shape_old = x_price.shape
x_shape_new_1 = x_shape_old[0]
x_shape_new_3 = len(rate_columns)
x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
x_price.shape

(3602, 60, 5)

In [16]:
y_close = dt2['open_rate'].to_numpy()[N+1:]
y_close.shape

(3602,)

In [17]:
# 裁剪，成整数批次
_x_y_len = int(x_price.shape[0]/batch_size) * batch_size

x_price = x_price[-_x_y_len:]
y_close = y_close[-_x_y_len:]
x_price = x_price.reshape(x_price.shape[0]//batch_size, batch_size, x_price.shape[1], x_price.shape[2])
y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)
print(x_price.shape)
print(y_close.shape)

(225, 16, 60, 5)
(225, 16, 1)


In [18]:
# 这里要区分训练集和测试集



dataset = Data.TensorDataset(torch.FloatTensor(x_price), torch.FloatTensor(y_close))
batch_size = 16
train_loader, test_loader = Data.random_split(dataset,
                                             lengths=[int(0.9*len(dataset)), len(dataset)- int(0.9*len(dataset))],
                                             generator=torch.Generator().manual_seed(0))

# 神经网络

In [19]:
# 这里做一个网络
import torch.nn as nn
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.gru(x) # gru层
        out = self.fc(out[:,-1, :]) # 全连接层
        return out

# 训练

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = GRUModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = torch.nn.MSELoss() # 均方差误差函数
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):  # 迭代一次
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # 获取输入
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 梯度置 0
        optimizer.zero_grad()
        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 打印状态信息
        running_loss += loss.item()
        # if i % 200 == 199:    # 每 200 批次打印一次
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 200))
        #     running_loss = 0.0
    # 每轮迭代打印一次
    if epoch % 10 == 0:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
        running_loss = 0.0

    
print('Finished Training.')

[1,   202] loss: nan
[11,   202] loss: nan
[21,   202] loss: nan
[31,   202] loss: nan
[41,   202] loss: nan
[51,   202] loss: nan
[61,   202] loss: nan
[71,   202] loss: nan
[81,   202] loss: nan
